<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод</a></span><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Классификация токсичных комментариев

Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Построить модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import re
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import nltk
from nltk.corpus import stopwords as nltk_stopwords
import spacy
from scipy.sparse import vstack

nltk.download('stopwords');

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [6]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [8]:
data.duplicated().sum()

0

In [9]:
data.isna().sum()

Unnamed: 0    0
text          0
toxic         0
dtype: int64

In [7]:
data.shape

(159292, 3)

In [8]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [9]:
def clear_text(text):
    text = " ".join(re.sub(r"[^a-zA-z' ]", ' ', text).split()).lower()
    return text

In [10]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [11]:
def lemmatize(sentence):
    sentence = clear_text(sentence)
    doc = nlp(sentence)
    return " ".join([token.lemma_ for token in doc])

In [12]:
corpus = data['text'].values
print("Исходный текст:", corpus[0])
print("Лемматизированный текст:", lemmatize(corpus[0]))

Исходный текст: Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27
Лемматизированный текст: explanation why the edit make under my username hardcore metallica fan be revert they be not vandalism just closure on some gas after I vote at new york doll fac and please do not remove the template from the talk page since I be retire now


In [13]:
%%time
data['lemm_text'] = data['text'].apply(lemmatize)
data.head()

CPU times: user 17min 33s, sys: 2.65 s, total: 17min 36s
Wall time: 17min 37s


,Unnamed: 0,text,toxic,lemm_text
0,0,Explanation\nWhy the edits made under my usern...,0,explanation why the edit make under my usernam...
1,1,D'aww! He matches this background colour I'm s...,0,d'aww he match this background colour I be see...
2,2,"Hey man, I'm really not trying to edit war. It...",0,hey man I be really not try to edit war it be ...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,more I can not make any real suggestion on imp...
4,4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


In [14]:
for i in range(3):
    print("Исходный текст:\n", data['text'][i], '\n')
    print("Очищенный и лемматизированный текст:\n", data['lemm_text'][i], '\n')

Исходный текст:
 Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27 

Очищенный и лемматизированный текст:
 explanation why the edit make under my username hardcore metallica fan be revert they be not vandalism just closure on some gas after I vote at new york doll fac and please do not remove the template from the talk page since I be retire now 

Исходный текст:
 D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC) 

Очищенный и лемматизированный текст:
 d'aww he match this background colour I be seemingly stuck with thank talk january utc 

Исходный текст:
 Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my t

In [15]:
features = data['lemm_text'].values
target = data['toxic']

features_learn, features_test, target_learn, target_test = train_test_split(
    features, target, test_size=0.20, random_state=12345)
features_train, features_valid, target_train, target_valid = train_test_split(
    features_learn, target_learn, test_size=0.25, random_state=12345)

print('Размеры и размерности выборок:', features_train.shape, features_valid.shape, features_test.shape)

Размеры и размерности выборок: (95574,) (31859,) (31859,)


In [16]:
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

features_train = count_tf_idf.fit_transform(features_train)
features_valid = count_tf_idf.transform(features_valid)
features_test = count_tf_idf.transform(features_test)

features_learn = vstack([features_train, features_valid])

print('Размеры и размерности выборок:', features_train.shape, features_valid.shape, features_test.shape)

Размеры и размерности выборок: (95574, 113332) (31859, 113332) (31859, 113332)


        Данные считаны и записаны в переменную data. Пропусков и дубликатов не обнаружено. Текст очищен и лемматизирован.
     В качестве обучающих признаков взято значение TFIDF. Исследован баланс классов: большая часть данных в столбце 
     'toxic' равна 0, необходимо провести баллансирование классов. Данные разбиты на 3 выборки в соотношении 3:1:1.

## Обучение

In [18]:
%%time
best_f1 = 0
for c in range(5, 16, 1):
    for penalty in ['l1', 'l2']:
        model = LogisticRegression(random_state=12345, solver='liblinear', C=c,
                                   max_iter=300, penalty=penalty, class_weight='balanced')
        model.fit(features_train, target_train)
        predict = model.predict(features_valid)
        f1 = f1_score(target_valid, predict)
        if f1 > best_f1:
            best_f1 = f1
            best_c = c
            best_penalty = penalty
        print(f'Значение C: {c} Значение penalty: {penalty} Значение F1: {f1}')
print(f'Лучшее значение C: {best_c} Лучшее значение penalty: {best_penalty} Лучшее значение F1: {best_f1:.3f}')

Значение C: 5 Значение penalty: l1 Значение F1: 0.7522008947900131
Значение C: 5 Значение penalty: l2 Значение F1: 0.7545197740112993
Значение C: 6 Значение penalty: l1 Значение F1: 0.7509778357235986
Значение C: 6 Значение penalty: l2 Значение F1: 0.7552904416986224
Значение C: 7 Значение penalty: l1 Значение F1: 0.7505100553774411
Значение C: 7 Значение penalty: l2 Значение F1: 0.7551689719093111
Значение C: 8 Значение penalty: l1 Значение F1: 0.7503287070854638
Значение C: 8 Значение penalty: l2 Значение F1: 0.7544738725841088
Значение C: 9 Значение penalty: l1 Значение F1: 0.7508802816901408
Значение C: 9 Значение penalty: l2 Значение F1: 0.7543708799082832
Значение C: 10 Значение penalty: l1 Значение F1: 0.7481296758104737
Значение C: 10 Значение penalty: l2 Значение F1: 0.7533026995979321
Значение C: 11 Значение penalty: l1 Значение F1: 0.7482014388489209
Значение C: 11 Значение penalty: l2 Значение F1: 0.7529513389000864
Значение C: 12 Значение penalty: l1 Значение F1: 0.7456978

In [19]:
%%time
best_f1 = 0
for depth in [None] + [i for i in range(5, 7)]:
    for iterations in range(300, 500, 100):
        model = CatBoostClassifier(depth=depth, learning_rate=0.2, iterations=iterations, silent=True, random_state=12345)
        model.fit(features_train, target_train)
        predict = model.predict(features_valid)
        f1 = f1_score(target_valid, predict)
        if f1 > best_f1:
            best_f1 = f1
            best_depth = depth
            best_iterations = iterations
        print(f'Значение depth: {depth} Значение iterations: {iterations} Значение F1: {f1}')
print(f'Лучшее значение depth: {best_depth} Лучшее значение iterations: {best_iterations} Лучшее значение F1: {best_f1:.3f}')

Значение depth: None Значение iterations: 300 Значение F1: 0.7376246600181323
Значение depth: None Значение iterations: 400 Значение F1: 0.748294434470377
Значение depth: 5 Значение iterations: 300 Значение F1: 0.732188527584947
Значение depth: 5 Значение iterations: 400 Значение F1: 0.7411296162201304
Значение depth: 6 Значение iterations: 300 Значение F1: 0.7376246600181323
Значение depth: 6 Значение iterations: 400 Значение F1: 0.748294434470377
Лучшее значение depth: None Лучшее значение iterations: 400 Лучшее значение F1: 0.748
CPU times: user 57min 35s, sys: 29.7 s, total: 58min 5s
Wall time: 58min 18s


        Изучено две модели: LogisticRegression и CatBoostClassifier. Для моделей найдены лучшие параметры. Лучший результат
     дала модель LogisticRegression, значение f1 на валидационной выборке равно 0,755.

## Выводы

In [21]:
model = LogisticRegression(random_state=12345, solver='liblinear', C=6, max_iter=300, penalty='l2', class_weight='balanced')
model.fit(features_train, target_train)
predict = model.predict(features_test)

f1 = f1_score(target_test, predict)
print(f'Значение F1: {f1:.3f}')

Значение F1: 0.772


In [19]:
dummy_class = DummyClassifier(strategy="uniform", random_state=12345)
dummy_class.fit(features_learn, target_learn)
dummy_predict = dummy_class.predict(features_test)
dummy_f1 = f1_score(target_test, dummy_predict)
print(f'Значение F1 для модели DummyClassifier: {dummy_f1:.3f}')

Значение F1 для константной модели: 0.168


        Модель LogisticRegression с лучшими параметрами обучена на обучающей выборке и протестирована на
     тестовой, значение f1 равно 0.772. Проведена проверка на адекватность с помощью модели DummyClassifier.

## Общий вывод
### Подготовка
    - Данные считаны и записаны в переменную data;
    - Пропусков и дубликатов не обнаружено;
    - Текст очищен и лемматизирован;
    - В качестве обучающих признаков взято значение TFIDF;
    - Исследован баланс классов: большая часть данных в столбце 'toxic' равна 0, необходимо провести баллансирование 
    классов;
    - Данные разбиты на 3 выборки в соотношении 3:1:1.
    
### Обучение
    - Изучено две модели: LogisticRegression и CatBoostClassifier;
    - Для моделей найдены лучшие параметры;
    - Лучший результат дала модель LogisticRegression, значение f1 на валидационной выборке равно 0,755.
### Выводы
    - Модель LogisticRegression с лучшими параметрами обучена на обучающей выборке;
    - Модель была протестирована на тестовой, значение f1 равно 0,772;
    - Проведена проверка на адекватность с помощью модели DummyClassifier.